In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
from config import DEVICE, SEED, MODEL_CONFIG, TRAINING_CONFIG, DATASET_CONFIG
from model import JointCausalModel
from utility import compute_class_weights, label_value_counts
from dataset_collator import CausalDataset, CausalDatasetCollator
from config import id2label_cls, id2label_bio, id2label_rel
from evaluate_joint_causal_model import evaluate_model, print_eval_report
from trainer import train_model
import random

c:\Users\norouzin\Desktop\JointLearning\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data_path = "C:\\Users\\norouzin\\Desktop\\JointLearning\\datasets\\expert_multi_task_data\\train.csv"
val_data_path = "C:\\Users\\norouzin\\Desktop\\JointLearning\\datasets\\expert_multi_task_data\\val.csv"
test_data_path = "C:\\Users\\norouzin\\Desktop\\JointLearning\\datasets\\expert_multi_task_data\\test.csv"
train_df = pd.read_csv(train_data_path)
val_df = pd.read_csv(val_data_path)
test_df = pd.read_csv(test_data_path)

In [3]:
train_dataset = CausalDataset(
    train_df,
    tokenizer_name='bert-base-uncased',
    max_length=DATASET_CONFIG["max_length"],
)
# %%
val_dataset = CausalDataset(
    val_df,
    tokenizer_name='bert-base-uncased',
    max_length=DATASET_CONFIG["max_length"],
)
# %%
test_dataset = CausalDataset(
    test_df,
    tokenizer_name='bert-base-uncased',
    max_length=DATASET_CONFIG["max_length"],
)
# %%

In [4]:
labels_flat = label_value_counts(train_dataset)
# %%
cls_label_flat = labels_flat["cls_labels_flat"]
bio_label_flat = labels_flat["bio_labels_flat"]
rel_label_flat = labels_flat["rel_labels_flat"]
# %%
# Calculate class weights
cls_weights = compute_class_weights(labels_list=cls_label_flat, num_classes=MODEL_CONFIG["num_cls_labels"], technique="ens", ignore_index=-100)
bio_weights = compute_class_weights(labels_list=bio_label_flat, num_classes=MODEL_CONFIG["num_bio_labels"], technique="ens", ignore_index=-100)
rel_weights = compute_class_weights(labels_list=rel_label_flat, num_classes=MODEL_CONFIG["num_rel_labels"], technique="ens", ignore_index=-100)
print(f"CLS Weights: {cls_weights}")
print(f"BIO Weights: {bio_weights}")
print(f"REL Weights: {rel_weights}")
# %%

cls_labels_value_counts:
 0    1047
1    1035
Name: count, dtype: int64
bio_labels_value_counts:
  6      52764
 3       8717
 1       6948
-100     4164
 2       1320
 0       1179
 5        483
 4         79
Name: count, dtype: int64
rel_labels_value_counts:
 0    2887
1    1494
Name: count, dtype: int64
CLS Weights: tensor([0.0015, 0.0016])
BIO Weights: tensor([0.0014, 0.0010, 0.0014, 0.0010, 0.0132, 0.0026, 0.0010])
REL Weights: tensor([0.0011, 0.0013])


In [5]:
collator = CausalDatasetCollator(
    tokenizer=train_dataset.tokenizer
)
# %%
# take a 100 samples from train_dataset
train_dataset = torch.utils.data.Subset(train_dataset, random.sample(range(len(train_dataset)), 20))
val_dataset = torch.utils.data.Subset(val_dataset, random.sample(range(len(val_dataset)), 20))
# %%
train_dataloader = DataLoader(
    train_dataset,
    batch_size=TRAINING_CONFIG["batch_size"],
    collate_fn=collator,
    shuffle=True

)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=TRAINING_CONFIG["batch_size"],
    collate_fn=collator,
    shuffle=False
)

In [6]:
model = JointCausalModel(
    encoder_name=MODEL_CONFIG["encoder_name"],
    num_cls_labels=MODEL_CONFIG["num_cls_labels"],
    num_bio_labels=MODEL_CONFIG["num_bio_labels"],
    num_rel_labels=MODEL_CONFIG["num_rel_labels"],
    dropout=MODEL_CONFIG["dropout"]
)


In [7]:
optimizer = optim.AdamW(
    model.parameters(),
    lr=TRAINING_CONFIG["learning_rate"],
    weight_decay=TRAINING_CONFIG["weight_decay"]
)
# %%
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=2
)

In [10]:
model_save_path = r"C:\Users\norouzin\Desktop\JointLearning\src\jointlearning\tesbert\best_bert_softmax_model.pt"

In [11]:
trained_model, training_history = train_model(
        model=model,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        optimizer=optimizer,
        num_epochs=2,
        device=DEVICE,
        id2label_cls=id2label_cls,
        id2label_bio=id2label_bio,
        id2label_rel=id2label_rel,
        model_save_path=model_save_path,
        scheduler=scheduler,
        cls_class_weights=cls_weights,
        bio_class_weights=bio_weights, # Only for softmax
        rel_class_weights=rel_weights,
        patience_epochs=TRAINING_CONFIG["num_epochs"],
        seed=SEED,
        max_grad_norm=TRAINING_CONFIG["gradient_clip_val"],
        eval_fn_metrics=evaluate_model, # Pass your evaluate_model function here
        print_report_fn=print_eval_report # Pass your print_eval_report function here
    )


Epoch 1/2 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           4.1984
  Average Validation Loss:         2.7491
  Overall Validation Avg F1 (Macro): 0.4061
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.6491
    Macro Precision: 0.6515
    Macro Recall:    0.6500
    Accuracy:        0.6500
    Per-class details:
      non-causal  : F1=0.6316 (P=0.6667, R=0.6000, Support=10.0)
      causal      : F1=0.6667 (P=0.6364, R=0.7000, Support=10.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.1692
    Macro Precision: 0.3400
    Macro Recall:    0.2023
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.000 (P=0.000, R=0.000, S=12.0)
      I-C       : F1=0.000 (P=0.000, R=0.000, S=67.0)
      B-E       : F1=0.000 (P=0.000, R=0.000, S=11


Epoch 2/2 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           3.7879
  Average Validation Loss:         2.6883
  Overall Validation Avg F1 (Macro): 0.3496
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.4872
    Macro Precision: 0.5980
    Macro Recall:    0.5500
    Accuracy:        0.5500
    Per-class details:
      non-causal  : F1=0.3077 (P=0.6667, R=0.2000, Support=10.0)
      causal      : F1=0.6667 (P=0.5294, R=0.9000, Support=10.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.1807
    Macro Precision: 0.2137
    Macro Recall:    0.2067
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.000 (P=0.000, R=0.000, S=12.0)
      I-C       : F1=0.000 (P=0.000, R=0.000, S=67.0)
      B-E       : F1=0.000 (P=0.000, R=0.000, S=11

In [17]:
trained_model.save_pretrained(
    r"C:\Users\norouzin\Desktop\JointLearning\src\jointlearning\tesbert\HF_output"
)

In [19]:
tokenizer = train_dataset.dataset.tokenizer
tokenizer.save_pretrained(
    r"C:\Users\norouzin\Desktop\JointLearning\src\jointlearning\tesbert\HF_output"
)

('C:\\Users\\norouzin\\Desktop\\JointLearning\\src\\jointlearning\\tesbert\\HF_output\\tokenizer_config.json',
 'C:\\Users\\norouzin\\Desktop\\JointLearning\\src\\jointlearning\\tesbert\\HF_output\\special_tokens_map.json',
 'C:\\Users\\norouzin\\Desktop\\JointLearning\\src\\jointlearning\\tesbert\\HF_output\\vocab.txt',
 'C:\\Users\\norouzin\\Desktop\\JointLearning\\src\\jointlearning\\tesbert\\HF_output\\added_tokens.json',
 'C:\\Users\\norouzin\\Desktop\\JointLearning\\src\\jointlearning\\tesbert\\HF_output\\tokenizer.json')